# Testing Suites

In [2]:
using Test
using LinearAlgebra
using FrankWolfe
using Suppressor
include("auxiliary_functions.jl")
include("terms_and_polynomials.jl")
include("objective_functions.jl")
include("oracle_avi.jl");
include("auxiliary_functions_avi.jl");

**Tests for auxiliary functions**

In [18]:
matrix = vecvec_to_mat([[1, 2, 3, 4, 3], [0, 1, 2, 3, 2], [1, 3, 1, 2, 1]])
matrix_sorted = vecvec_to_mat([[1, 3, 3, 2, 4], [0, 2, 2, 1, 3], [1, 1, 1, 3, 2]])
matrix_unique = vecvec_to_mat([[1, 3, 2, 4], [0, 2, 1, 3], [1, 1, 3, 2]]);

In [4]:
@testset "Test suite for get_unique_columns" begin
    mat1_unique, mat2_unique, _ = get_unique_columns(matrix, matrix)
    @test mat1_unique == matrix_unique
    @test mat2_unique == matrix_unique
end;

Test Summary:                     | Pass  Total  Time
Test suite for get_unique_columns |    2      2  3.1s


In [5]:
@testset "Test suite for deg_lex_sort" begin
    matrix_sorted_1, matrix_sorted_2, a = deg_lex_sort(matrix, 1. * matrix)
    @test matrix_sorted_1 == matrix_sorted
    @test matrix_sorted_2 == matrix_sorted
end;

Test Summary:               | Pass  Total  Time
Test suite for deg_lex_sort |    2      2  0.3s


In [6]:
@testset "Test suite for vecvec_to_mat" begin
  vecvec1 = [[1, 2, 3], [4, 5, 6]]
  vecvec2 = [[1, 2], [3, 4], [5, 6]]

  row_1 = reshape([1, 2, 3], 1, 3)
  row_2 = reshape([4, 5, 6], 1, 3)
  
  mat1 = vcat(row_1, row_2)
  mat2 = reshape([1, 2, 3, 4, 5, 6], 2, 3)

  @test vecvec_to_mat(vecvec1) == mat1
  @test vecvec_to_mat(vecvec2, arr_is_col=1) == mat2
end;

Test Summary:                | Pass  Total  Time
Test suite for vecvec_to_mat |    2      2  0.1s


In [7]:
@testset "Test suite for compute_degree" begin
  @test compute_degree(matrix) == reshape([2, 6, 6, 9, 6], 1, 5)
end;

Test Summary:                 | Pass  Total  Time
Test suite for compute_degree |    1      1  0.0s


**Tests for oracle_avi**

In [19]:
@suppress_err @testset "Test suite for fit" begin
    for oracle in ["CG", "PCG", "BPCG"]
        m, n = rand(15:25), rand(4:10)
        X_train = rand(m, n)
        X_train_transformed, sets = fit(X_train,oracle_type=oracle)
        loss_list = Vector{Float64}([])
        for col in 1:size(sets.G_evaluations, 2)
            cur_col = sets.G_evaluations[:, col]
            loss = 1 / size(cur_col, 1) * (norm(cur_col, 2)^2)
            loss_list = append!(loss_list, loss)
        end
        @test all(loss_list .<= 0.1)
    end
end;

Test Summary:      | Pass  Total   Time
Test suite for fit |    3      3  35.4s


In [17]:
@testset "Test suite for l1_projection" begin
  vec1 = rand(1:10, 20)
  radius_1 = 2.5
  radius_2 = 3.0
  @test norm(l1_projection(vec1), 1) ≈ 1
  @test norm(l1_projection(vec1, radius=radius_1), 1) ≈ radius_1
  @test norm(l1_projection(vec1, radius=radius_2), 1) ≈ radius_2
end;

Test Summary:                | Pass  Total  Time
Test suite for l1_projection |    3      3  0.0s


**Tests for terms_and_polynomials**

In [21]:
@testset "Test suite for purge" begin
    matrix_2 = vecvec_to_mat([[1, 2, 3], 
                              [0, 0, 1],
                              [1, 1, 1]])
    
    matrix_1 = vecvec_to_mat([[1, 1],
                              [0, 1],
                              [0, 1]])
    
    matrix_2_purged, matrix_2_purged_2, _ = purge(matrix_2, 1. * matrix_2, matrix_1)
    
    @test size(matrix_2_purged, 2) == 0
    @test size(matrix_2_purged_2, 2) == 0
    
    matrix_2 = vecvec_to_mat([[1, 2, 3],
                             [0, 0, 1],
                             [1, 1, 1]])
    
    matrix_1 = vecvec_to_mat([[1, 1],
                             [0, 2],
                             [2, 1]])
    
    matrix_2_purged, matrix_2_purged_2, _ = purge(matrix_2, 1. * matrix_2, matrix_1)
    
    @test matrix_2_purged == matrix_2_purged_2
    @test matrix_2_purged == vecvec_to_mat([[1, 2, 3],
                                            [0, 0, 1],
                                            [1, 1, 1]])
    
end;

Test Summary:        | Pass  Total  Time
Test suite for purge |    4      4  0.0s


In [22]:
@testset "Test suite for update_coefficient_vectors" begin
    G_coefficient_vectors = reshape([1, 2, 0], 3, 1)
    
    vec1 = reshape([1, 2], 2, 1)
    
    G_coefficient_vectors = update_coefficient_vectors(G_coefficient_vectors, vec1)
    G_coefficient_vectors = vcat(G_coefficient_vectors, zeros(1, size(G_coefficient_vectors, 2)))
    G_coefficient_vectors = update_coefficient_vectors(G_coefficient_vectors, vec1)
    G_coefficient_vectors = vcat(G_coefficient_vectors, zeros(1, size(G_coefficient_vectors, 2)))
    G_coefficient_vectors = vcat(G_coefficient_vectors, zeros(1, size(G_coefficient_vectors, 2)))
    
    vec1 = reshape([1, 2, 3], 3, 1)
    
    G_coefficient_vectors = update_coefficient_vectors(G_coefficient_vectors, vec1)
    
    @test G_coefficient_vectors == vecvec_to_mat([[1., 1., 1., 1.],
                                                  [2., 0., 0., 0.],
                                                  [0., 2., 0., 0.],
                                                  [0., 0., 2., 0.],
                                                  [0., 0., 0., 2.],
                                                  [0., 0., 0., 3.]])
end;

Test Summary:                             | Pass  Total  Time
Test suite for update_coefficient_vectors |    1      1  2.0s


**Tests for L2Loss**

In [3]:
@testset "Test suite for 'evaluate_function' in L2Loss" begin
    for m in 1:5
        for n in 1:10
            A = rand(m, n)
            b = rand(m)
            x = rand(n)
            lambda = rand() * n
            _, evaluate_function, _ = L2Loss(A, b, lambda, A' * A, A' * b, b' * b)
            
            @test 1/m * norm(A * x + b, 2)^2 + lambda * norm(x, 2)^2 / 2 ≈ evaluate_function(x)
        end
    end
end;

Test Summary:                                | Pass  Total  Time
Test suite for 'evaluate_function' in L2Loss |   50     50  0.6s


In [4]:
@testset "Test suite for 'evaluate_gradient!' in L2Loss" begin
    for m in 1:5
        for n in 1:10
            A = rand(m, n)
            b = rand(m)
            x = rand(n)
            lambda = rand() * n
            _, _, evaluate_gradient! = L2Loss(A, b, lambda, A' * A, A' * b, b' * b)
            
            gradient = 2/m * (A' * A * x + A' * b + m/2 * lambda * x)
            approx_vec = (gradient .≈ evaluate_gradient!(zeros(n), x))
            @test all(approx_vec)
        end
    end
end;

Test Summary:                                 | Pass  Total  Time
Test suite for 'evaluate_gradient!' in L2Loss |   50     50  0.4s
